In [13]:
import gradio as gr
from PIL import Image, ImageDraw
import requests, io
import numpy as np
import os
from datetime import datetime
from gradio_image_annotation import image_annotator

# Custom Vision 설정
PREDICTION_KEY = "5k8oJDDDmqLn5Yy9n1Q16CHetW6H0pvTjFPj1Q4JpQl7dAVJE0WhJQQJ99BEACYeBjFXJ3w3AAAIACOGZmg4"
ENDPOINT_URL = "https://cv7934-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/92adf90f-3b67-4923-b2eb-1804da244279/detect/iterations/Iteration1/image"

# 감지 함수
def detect_with_boxes(image: Image.Image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    headers = {
        "Prediction-Key": PREDICTION_KEY,
        "Content-Type": "application/octet-stream"
    }
    response = requests.post(ENDPOINT_URL, headers=headers, data=buffered.getvalue())
    results = response.json()

    # 결과 그리기
    image_with_boxes = image.copy()
    draw = ImageDraw.Draw(image_with_boxes)
    for pred in results["predictions"]:
        if pred["probability"] > 0.5:
            w, h = image.width, image.height
            box = pred["boundingBox"]
            left = box["left"] * w
            top = box["top"] * h
            right = left + box["width"] * w
            bottom = top + box["height"] * h
            draw.rectangle([left, top, right, bottom], outline="red", width=2)
            draw.text((left, top), f"{pred['tagName']} ({pred['probability']:.2f})", fill="red")

    return image_with_boxes

# 업로드 → 결과 이미지 + 태깅용 원본 전달
def handle_image_upload(image: Image.Image):
    ai_result = detect_with_boxes(image)

    # numpy 배열 전달
    annotator_input = {
        "image": np.array(image.convert("RGB")),  # <== base64 ❌
        "annotations": []
    }

    return ai_result, annotator_input

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🚬 담배꽁초 감지 (AI vs 사용자)")

    image_input = gr.Image(type="pil", label="📤 이미지 업로드")

    with gr.Row():
        ai_output = gr.Image(label="🤖 AI 감지 결과")
        annotator = image_annotator(
            label_list=["사용자"],
            label_colors=[(0, 0, 255)],
            scale=False
        )

    image_input.change(
        fn=handle_image_upload,
        inputs=image_input,
        outputs=[ai_output, annotator]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


## 감지 후 태깅 결과 저장

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw
import requests, io, json
import numpy as np
from datetime import datetime
from gradio_image_annotation import image_annotator

# ─────────────────────────────────────
# Custom Vision 설정
# ─────────────────────────────────────
PREDICTION_KEY = "5k8oJDDDmqLn5Yy9n1Q16CHetW6H0pvTjFPj1Q4JpQl7dAVJE0WhJQQJ99BEACYeBjFXJ3w3AAAIACOGZmg4"
ENDPOINT_URL = "https://cv7934-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/92adf90f-3b67-4923-b2eb-1804da244279/detect/iterations/Iteration1/image"

# ─────────────────────────────────────
# AI 감지 함수
# ─────────────────────────────────────
def detect_with_boxes(image: Image.Image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    headers = {
        "Prediction-Key": PREDICTION_KEY,
        "Content-Type": "application/octet-stream"
    }
    response = requests.post(ENDPOINT_URL, headers=headers, data=buffered.getvalue())
    response.raise_for_status()
    results = response.json()

    image_with_boxes = image.copy()
    draw = ImageDraw.Draw(image_with_boxes)
    for pred in results["predictions"]:
        if pred["probability"] > 0.5:
            w, h = image.width, image.height
            box = pred["boundingBox"]
            left = box["left"] * w
            top = box["top"] * h
            right = left + box["width"] * w
            bottom = top + box["height"] * h
            draw.rectangle([left, top, right, bottom], outline="red", width=2)
            draw.text((left, top), f"{pred['tagName']} ({pred['probability']:.2f})", fill="red")

    return image_with_boxes

# ─────────────────────────────────────
# 이미지 업로드 + AI 감지 실행
# ─────────────────────────────────────
def handle_uploaded_image(image):
    if image is None:
        return None, None, None, gr.update(visible=False)

    ai_img = detect_with_boxes(image)
    annot_input = {
        "image": np.array(image.convert("RGB")),
        "boxes": []
    }

    return ai_img, annot_input, image, gr.update(visible=True)

# ─────────────────────────────────────
# 겹쳐 시각화
# ─────────────────────────────────────
def overlap_visualizer(base_img, user_data):
    if base_img is None or not user_data:
        return None

    img = base_img.copy()
    draw = ImageDraw.Draw(img)

    annotations = []
    if isinstance(user_data, dict) and "boxes" in user_data:
        annotations = user_data["boxes"]
    elif isinstance(user_data, list):
        annotations = user_data

    for box in annotations:
        x0, y0 = box.get("xmin"), box.get("ymin")
        x1, y1 = box.get("xmax"), box.get("ymax")
        label = box.get("label", "사용자")
        if None not in [x0, y0, x1, y1]:
            draw.rectangle([x0, y0, x1, y1], outline="blue", width=2)
            draw.text((x0, y0 - 10), label, fill="blue")

    return img

# ─────────────────────────────────────
# 저장 함수
# ─────────────────────────────────────
def save_annotations(annot_data):
    print("💾 저장 요청됨:", type(annot_data), annot_data)

    try:
        if isinstance(annot_data, dict) and "boxes" in annot_data:
            annotations = annot_data["boxes"]
        else:
            return f"❌ 'boxes' 키가 없거나 잘못된 구조입니다."

        if not annotations:
            return "❌ 박스 정보가 없습니다."

        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = f"annotation_{now}.json"
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump({"annotations": annotations}, f, ensure_ascii=False, indent=2)

        return f"✅ 저장 완료: {file_path}"

    except Exception as e:
        return f"❌ 저장 중 오류: {str(e)}"

# ─────────────────────────────────────
# Gradio UI
# ─────────────────────────────────────
with gr.Blocks() as demo:
    gr.Markdown("## 🚬 담배꽁초 감지 + 사용자 태깅 비교")

    with gr.Row():
        image_input = gr.Image(type="pil", label="📤 이미지 업로드")
    upload_btn = gr.Button("➡️ 감지 시작")

    with gr.Row(visible=False) as result_row:
        with gr.Column():
            ai_output = gr.Image(label="🤖 AI 감지 결과")
            overlap_output = gr.Image(label="🎯 겹쳐 보기 (AI + 사용자)")
        with gr.Column():
            annotator = image_annotator(
                label_list=["사용자"],
                label_colors=[(0, 0, 255)],
                scale=False
            )
            save_btn = gr.Button("💾 사용자 태깅 결과 저장")
            save_text = gr.Textbox(label="📁 저장 상태")

    # 감지 실행 버튼
    upload_btn.click(
        fn=handle_uploaded_image,
        inputs=image_input,
        outputs=[ai_output, annotator, image_input, result_row]
    )

    # 태깅 변경 시 자동 겹쳐 보기 업데이트
    annotator.change(
        fn=overlap_visualizer,
        inputs=[image_input, annotator],
        outputs=overlap_output
    )

    # 저장 버튼 클릭 시
    save_btn.click(
        fn=save_annotations,
        inputs=annotator,
        outputs=save_text
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


💾 저장 요청됨: <class 'dict'> {'image': array([[[ 5, 22, 18],
        [ 5, 22, 18],
        [ 6, 23, 19],
        ...,
        [ 5, 16, 15],
        [ 5, 16, 15],
        [ 5, 16, 15]],

       [[ 5, 22, 18],
        [ 5, 22, 18],
        [ 6, 23, 19],
        ...,
        [ 5, 16, 15],
        [ 5, 16, 15],
        [ 5, 16, 15]],

       [[ 5, 22, 18],
        [ 5, 22, 18],
        [ 6, 23, 19],
        ...,
        [ 5, 16, 15],
        [ 5, 16, 15],
        [ 5, 16, 15]],

       ...,

       [[ 4, 41, 26],
        [ 4, 41, 26],
        [ 4, 41, 26],
        ...,
        [38, 64, 60],
        [39, 65, 63],
        [39, 65, 63]],

       [[ 4, 41, 24],
        [ 4, 41, 24],
        [ 4, 41, 24],
        ...,
        [39, 65, 61],
        [39, 65, 61],
        [39, 65, 61]],

       [[ 4, 41, 24],
        [ 4, 41, 24],
        [ 4, 41, 24],
        ...,
        [39, 66, 59],
        [39, 65, 61],
        [39, 65, 61]]], shape=(700, 700, 3), dtype=uint8), 'boxes': [{'label': '사용자', 'color':

## AI와 태깅 결과 비교

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw
import numpy as np
import io
import requests
from gradio_image_annotation import image_annotator

# 📌 Custom Vision API 설정
PREDICTION_KEY = "5k8oJDDDmqLn5Yy9n1Q16CHetW6H0pvTjFPj1Q4JpQl7dAVJE0WhJQQJ99BEACYeBjFXJ3w3AAAIACOGZmg4"
ENDPOINT_URL = "https://cv7934-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/92adf90f-3b67-4923-b2eb-1804da244279/detect/iterations/Iteration1/image"

# 🔍 AI 감지 함수
def detect_with_boxes(image: Image.Image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    headers = {
        "Prediction-Key": PREDICTION_KEY,
        "Content-Type": "application/octet-stream"
    }
    response = requests.post(ENDPOINT_URL, headers=headers, data=buffered.getvalue())
    results = response.json()

    ai_boxes = []
    image_with_boxes = image.copy()
    draw = ImageDraw.Draw(image_with_boxes)

    for pred in results["predictions"]:
        if pred["probability"] > 0.5:
            w, h = image.width, image.height
            box = pred["boundingBox"]
            left = int(box["left"] * w)
            top = int(box["top"] * h)
            right = int((box["left"] + box["width"]) * w)
            bottom = int((box["top"] + box["height"]) * h)

            ai_boxes.append({
                "label": pred["tagName"],
                "xmin": left,
                "ymin": top,
                "xmax": right,
                "ymax": bottom
            })

            draw.rectangle([left, top, right, bottom], outline="red", width=2)
            draw.text((left, top), f"{pred['tagName']} ({pred['probability']:.2f})", fill="red")

    return image_with_boxes, ai_boxes

# 📦 IoU 계산 함수
def calculate_iou(boxA, boxB):
    xA = max(boxA["xmin"], boxB["xmin"])
    yA = max(boxA["ymin"], boxB["ymin"])
    xB = min(boxA["xmax"], boxB["xmax"])
    yB = min(boxA["ymax"], boxB["ymax"])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA["xmax"] - boxA["xmin"]) * (boxA["ymax"] - boxA["ymin"])
    boxBArea = (boxB["xmax"] - boxB["xmin"]) * (boxB["ymax"] - boxB["ymin"])
    unionArea = float(boxAArea + boxBArea - interArea)

    if unionArea == 0:
        return 0.0
    else:
        return interArea / unionArea

# 📤 업로드 처리
def handle_upload(image: Image.Image):
    ai_image, ai_boxes = detect_with_boxes(image)

    annotator_input = {
        "image": np.array(image.convert("RGB")),
        "annotations": []
    }

    return ai_image, annotator_input, ai_boxes

# ✅ IoU 비교
def compare_iou(user_data, ai_boxes):
    if not user_data or "boxes" not in user_data:
        return "❌ 사용자 태깅 없음"

    user_boxes = user_data["boxes"]
    if not user_boxes or not ai_boxes:
        return "❌ 비교할 박스 없음"

    ious = []
    for user_box in user_boxes:
        user = {
            "xmin": user_box["xmin"],
            "ymin": user_box["ymin"],
            "xmax": user_box["xmax"],
            "ymax": user_box["ymax"]
        }
        best_iou = max([calculate_iou(user, ai_box) for ai_box in ai_boxes])
        ious.append(best_iou)

    avg_iou = sum(ious) / len(ious)
    return f"✅ 평균 IoU: {avg_iou:.2f}"

# 🎛️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🚬 담배꽁초 감지: AI vs 사용자 태깅")

    image_input = gr.Image(type="pil", label="이미지 업로드")
    upload_btn = gr.Button("AI 감지 및 태깅 시작")

    with gr.Row(visible=False) as result_row:
        ai_result = gr.Image(label="🤖 AI 감지 결과 (빨간 박스)")
        annotator = image_annotator(label_list=["bungee"], label_colors=[(0, 0, 255)])

    hidden_ai_boxes = gr.State([])  # AI 박스 저장용
    compare_btn = gr.Button("📐 IoU 비교")
    iou_text = gr.Textbox(label="결과")

    upload_btn.click(fn=handle_upload, inputs=image_input, outputs=[ai_result, annotator, hidden_ai_boxes])
    upload_btn.click(lambda: gr.update(visible=True), None, result_row)

    compare_btn.click(fn=compare_iou, inputs=[annotator, hidden_ai_boxes], outputs=iou_text)

demo.launch()


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import gradio as gr
from PIL import Image, ImageDraw
import numpy as np
import io
import requests
import json
import os
from gradio_image_annotation import image_annotator
#──────────────────────────────────────────────────────────────
# Custom Vision API 설정
#──────────────────────────────────────────────────────────────
PREDICTION_KEY = "5k8oJDDDmqLn5Yy9n1Q16CHetW6H0pvTjFPj1Q4JpQl7dAVJE0WhJQQJ99BEACYeBjFXJ3w3AAAIACOGZmg4"
ENDPOINT_URL = "https://cv7934-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/92adf90f-3b67-4923-b2eb-1804da244279/detect/iterations/Iteration1/image"
#──────────────────────────────────────────────────────────────
# IoU 계산 함수
#──────────────────────────────────────────────────────────────
def calculate_iou(boxA, boxB):
    xA = max(boxA["xmin"], boxB["xmin"])
    yA = max(boxA["ymin"], boxB["ymin"])
    xB = min(boxA["xmax"], boxB["xmax"])
    yB = min(boxA["ymax"], boxB["ymax"])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    unionArea = float(
        (boxA["xmax"] - boxA["xmin"]) * (boxA["ymax"] - boxA["ymin"]) +
        (boxB["xmax"] - boxB["xmin"]) * (boxB["ymax"] - boxB["ymin"]) - interArea
    )
    return interArea / unionArea if unionArea != 0 else 0
#──────────────────────────────────────────────────────────────
# AI 감지
#──────────────────────────────────────────────────────────────
def detect_with_boxes(image: Image.Image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    headers = {
        "Prediction-Key": PREDICTION_KEY,
        "Content-Type": "application/octet-stream"
    }
    response = requests.post(ENDPOINT_URL, headers=headers, data=buffered.getvalue())
    results = response.json()

    ai_boxes = []
    image_with_boxes = image.copy()
    draw = ImageDraw.Draw(image_with_boxes)

    for pred in results["predictions"]:
        if pred["probability"] > 0.5:
            w, h = image.width, image.height
            box = pred["boundingBox"]
            left = int(box["left"] * w)
            top = int(box["top"] * h)
            right = int((box["left"] + box["width"]) * w)
            bottom = int((box["top"] + box["height"]) * h)

            ai_boxes.append({
                "label": pred["tagName"],
                "xmin": left,
                "ymin": top,
                "xmax": right,
                "ymax": bottom
            })

            draw.rectangle([left, top, right, bottom], outline="red", width=2)
            draw.text((left, top), f"{pred['tagName']} ({pred['probability']:.2f})", fill="red")

    return image_with_boxes, ai_boxes
#──────────────────────────────────────────────────────────────
# 업로드 처리
#──────────────────────────────────────────────────────────────
def handle_upload(image: Image.Image):
    ai_img, ai_boxes = detect_with_boxes(image)
    annotator_input = {
        "image": np.array(image.convert("RGB")),
        "annotations": []
    }
    return ai_img, annotator_input, ai_boxes, image
#──────────────────────────────────────────────────────────────
# 박스 비교 및 시각화
#──────────────────────────────────────────────────────────────
def compare_boxes(user_data, ai_boxes):
    if not user_data or "boxes" not in user_data:
        return "❌ 사용자 태깅 없음", None, []

    img_array = user_data["image"]
    user_boxes = user_data["boxes"]
    img = Image.fromarray(img_array)
    draw = ImageDraw.Draw(img)

    matched_count = 0
    results_to_save = []
    used_ai = set()
    used_user = set()

    for u_idx, ubox in enumerate(user_boxes):
        user = {
            "xmin": ubox["xmin"],
            "ymin": ubox["ymin"],
            "xmax": ubox["xmax"],
            "ymax": ubox["ymax"]
        }

        best_iou = 0
        matched_ai_idx = -1
        for i, abox in enumerate(ai_boxes):
            iou = calculate_iou(user, abox)
            if iou > best_iou:
                best_iou = iou
                matched_ai_idx = i

        if best_iou >= 0.5:
            matched_count += 1
            used_ai.add(matched_ai_idx)
            used_user.add(u_idx)
            draw.rectangle([user["xmin"], user["ymin"], user["xmax"], user["ymax"]], outline="green", width=2)
        else:
            draw.rectangle([user["xmin"], user["ymin"], user["xmax"], user["ymax"]], outline="yellow", width=2)

        results_to_save.append({
            "label": ubox["label"],
            "xmin": ubox["xmin"],
            "ymin": ubox["ymin"],
            "xmax": ubox["xmax"],
            "ymax": ubox["ymax"],
            "matched": best_iou >= 0.5,
            "iou": round(best_iou, 2)
        })

    for idx, abox in enumerate(ai_boxes):
        if idx not in used_ai:
            draw.rectangle([abox["xmin"], abox["ymin"], abox["xmax"], abox["ymax"]], outline="orange", width=2)

    user_only = len(user_boxes) - matched_count
    ai_only = len(ai_boxes) - len(used_ai)

    # 점수 계산
    score_match = matched_count * 0.5
    score_user = user_only * 0.3
    score_ai = ai_only * 0.2
    total_score = score_match + score_user + score_ai

    msg = (
        f"✅ 비교 완료!\n"
        f"- 일치한 태그: {matched_count}/{len(user_boxes)}개\n"
        f"- 사용자만 태깅한 박스: {user_only}개\n"
        f"- AI만 감지한 박스: {ai_only}개\n"
        f"\n"
        f"📊 총점: {total_score:.1f}점 (일치: {score_match:.1f}, 사용자만: {score_user:.1f}, AI만: {score_ai:.1f})"
    )

    return msg, img, results_to_save
#──────────────────────────────────────────────────────────────
# 결과 저장
#──────────────────────────────────────────────────────────────
def save_results(image: Image.Image, results_to_save):
    os.makedirs("saved_images", exist_ok=True)
    filename = f"saved_images/image_{np.random.randint(100000)}.jpg"
    image.save(filename)

    with open("saved_annotations.json", "a", encoding="utf-8") as f:
        json.dump({"image": filename, "annotations": results_to_save}, f, ensure_ascii=False)
        f.write("\n")

    return f"💾 저장 완료: {filename}"
#──────────────────────────────────────────────────────────────
# Gradio UI
#──────────────────────────────────────────────────────────────
with gr.Blocks() as demo:
    gr.Markdown("## 🧪 담배꽁초 감지 비교 (사용자 vs AI)")

    image_input = gr.Image(type="pil", label="이미지 업로드")
    start_btn = gr.Button("🟦 AI 감지 및 태깅 시작")

    with gr.Row(visible=False) as tag_row:
        ai_result = gr.Image(label="🤖 AI 감지 결과")
        annotator = image_annotator(label_list=["bungee"], label_colors=[(0, 0, 255)])

    compare_btn = gr.Button("📐 비교", visible=False)

    with gr.Row(visible=False) as compare_row:
        compare_result = gr.Image(label="📊 사용자 vs AI 비교 결과")

    compare_text = gr.Textbox(label="결과 메시지", visible=False, lines=6)
    save_btn = gr.Button("💾 결과 저장", visible=False)
    save_text = gr.Textbox(label="저장 메시지", visible=False)

    hidden_ai_boxes = gr.State()
    original_image = gr.State()
    temp_save_result = gr.State()

    start_btn.click(fn=handle_upload,
                    inputs=image_input,
                    outputs=[ai_result, annotator, hidden_ai_boxes, original_image])
    start_btn.click(lambda: (gr.update(visible=True),)*2,
                    None, [tag_row, compare_btn])

    compare_btn.click(fn=compare_boxes,
                      inputs=[annotator, hidden_ai_boxes],
                      outputs=[compare_text, compare_result, temp_save_result])
    compare_btn.click(lambda: (gr.update(visible=True),)*3,
                      None, [compare_text, compare_row, save_btn])

    save_btn.click(fn=save_results,
                   inputs=[original_image, temp_save_result],
                   outputs=save_text)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
